# Подготовка данных






In [ ]:
# Переименование изображений

import os
root = ''
print("Введите полный путь до файлов: ")
root = input()
i = 0
print("Вы уверены, что правильно ввели путь и готовы запустить программу?\nВведите 1, если да, и 0, если нет")
answer = int(input())
if (answer == 1):
    print("Начало работы...\n")
    list = os.listdir(root)
    list.sort()

    for item in list:
      #for dir in listdir():
        #%cd '''{dir}/'''
        
        #for el in dir:
          fullpath = os.path.join(root, item)
          fullpath_replaced = os.path.join(root, str(i) + '.png')
        
          if((os.path.exists(fullpath_replaced) == False) and (fullpath_replaced != fullpath)):
              print(fullpath_replaced)
              os.rename(fullpath, fullpath_replaced)
              i += 1
          elif (fullpath_replaced == fullpath):
              i += 1
          else:
              for i in range(len(list) + 1):
                  fullpath_replaced = os.path.join(root, str(i) + '.png')
                  if (os.path.exists(fullpath_replaced) == False):
                      print(fullpath_replaced)
                      os.rename(fullpath, fullpath_replaced)
                      i = 0
                      break
                  else:
                      i += 1
        #%cd ..

    print("\nГотово!")
else:
    print("Завершение работы программы")

In [160]:
# Импортируем модули 

from PIL import Image
import multiprocessing, time, os
import matplotlib.pyplot as plt
from os import listdir

In [161]:
# Рескейлинг

def scale_transform(path):
  length = len(listdir(path))

  for i in range(length):
      img = Image.open(os.path.join(path + f'''{i}.png'''))
      img = img.resize((128,128), Image.ANTIALIAS)
      img.save(os.path.join(path + f'''{i}.png'''))

In [ ]:
# train_transform

scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/train/stop/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/train/break/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/train/before/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/train/main/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/train/up/')

In [ ]:
# test_transform

scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/test/stop/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/test/break/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/test/before/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/test/main/')
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/test/up/')

In [162]:
scale_transform('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/pred/buf/')

# Дополнительные функции

In [ ]:
# Импортируем модули

import shutil
import os

In [ ]:
# Соотносинение файла и его класса в словаре

def lists2dict(filenames, classes, path_len):
    for tmp in filenames:
        tmp = tmp[path_len:]
    dictic = {}
    i = 0
    for tmp in filenames:
        dictic[str2int(tmp)] = classes[i]
        i+=1
    dictic = sorted(dictic.items())
    return dictic


# Преобразование строки в число (номер картинки)

def str2int(str):
    tmp = str
    ret = []
    for i in tmp:
        if (i.isdigit()):
            ret.append(int(i))
    num = 0
    for i in range(len(ret)):
        num *= 10
        num += ret[i]
    return num


# Распределение по папкам

def cur_dir_sort(list, classes_dict, path, start_folder):
    classes = dict(zip(classes_dict.values(), classes_dict.keys()))
    if (os.path.exists(rf"{path}/answer") == False):
        os.mkdir(rf"{path}/answer")
    for tmp in list:
        print(path)
        if (os.path.exists(rf"{path}/answer/{classes.get(tmp[1])}") == False):
            os.mkdir(rf"{path}/answer/{classes.get(tmp[1])}")
        shutil.copyfile(f"{path}/{start_folder}/{tmp[0]}.png", f"{path}/answer/{classes.get(tmp[1])}/{tmp[0]}.png")
        

# Нейронка

In [196]:
# Импортируем модули

import tensorflow as tf
import numpy as np
import multiprocessing, time, os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

**Необходимые данные**

In [181]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/val'

img_width, img_height = 128, 128

# Размерность тензора на основе изобржения для входных данных в нейронную сеть
# backend Tensorflow, channels_Last
input_shape = (img_width, img_height, 3)

epochs = 10
batch_size = 10
nb_train_samples = 1325
nb_val_samples = 275

**CNN**

In [182]:
model = Sequential()

# Каскады свёртки и подвыборки (что-то типо разметки)
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Полносвязная сеть (классификация)
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('sigmoid'))

In [183]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

**Генерация изображений**

In [184]:
# Изменяем значение изображений от 0 до 1

datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Создаём генератор данных для обучения

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    save_format='png')

In [ ]:
# Создаём генератор данных для валидации

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    save_format='png')

**ОБУЧЕНИЕ**

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = val_generator,
    validation_steps = nb_val_samples // batch_size)

In [188]:
model.save('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/ExampleCNN.h5')

**Классификация знаков**

In [189]:
from keras.models import load_model

In [190]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/ExampleCNN.h5')

In [198]:
test_dir = '/content/drive/MyDrive/Colab Notebooks/CNN/SimpleCNN/test_data/prediction'

In [ ]:
# Создаём генератор данных для теста

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    save_format='png')

In [200]:
prediction = model.predict(test_generator, steps=len(test_generator), verbose=0)
# Get classes by max element in np (as a list)
classes = np.argmax(prediction, axis=1)
classes = np.ndarray.tolist(classes)
# Get filenames (set shuffle=false in generator is important)
filenames = test_generator.filenames

In [ ]:
print(filenames)

In [ ]:
print(classes)

In [ ]:
label_map = (train_generator.class_indices)
print(label_map)

In [ ]:
start_fold = 'buf'
listik = lists2dict(filenames, classes, len(start_fold))
cur_dir_sort(listik, label_map, test_dir, start_fold)